### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return np.sum(l_s**2 / l_c + r_s**2 / r_c, axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_f = l_s / l_c
        r_f = r_s / r_c
        return np.sum(l_s * np.log2(l_f, out=np.zeros_like(l_f), where=(l_f!=0)) +
                      r_s * np.log2(r_f, out=np.zeros_like(r_f), where=(r_f!=0)), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_c * np.max(l_s / l_c, axis=1) + r_c * np.max(r_s / r_c, axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x, y = self.__sort_samples(x, y)
        
        all_thresholds = (x[1:] + x[:-1]) / 2
        all_thresholds = all_thresholds[x[1:] != x[:-1]].reshape(-1, 1)
        if all_thresholds.size == 0:
            return [np.inf, np.inf]
        
        X = np.tile(x, (all_thresholds.size, 1))
        x_left_ind = (X > all_thresholds)        
        left_size = x_left_ind.sum(axis=1).reshape(-1, 1)
        right_size = x.shape[0] - left_size
        
        classes = np.tile(y + 1, (all_thresholds.size, 1))
        classes_left = np.zeros((all_thresholds.size, y.shape[0]), dtype=int)
        classes_left[x_left_ind] = classes[x_left_ind]
        classes_right = classes - classes_left
        
        classes_left = np.apply_along_axis(np.bincount, axis=1, 
                                           arr=classes_left, 
                                           minlength=(self.num_class+1))[:, 1:]
        classes_right = np.apply_along_axis(np.bincount, axis=1, 
                                           arr=classes_right, 
                                           minlength=(self.num_class+1))[:, 1:]
        G_values = self.G_function(left_size, classes_left,
                                   right_size, classes_right)
        if G_values.size == 0:
            return [np.inf, np.inf]
        
        threshold_id = np.argmax(G_values)
        G_max = G_values[threshold_id]        
        return G_max, all_thresholds[threshold_id]
        
    def __fit_node(self, x, y, node_id, depth):      
        classes = np.bincount(y)
        the_stoping_criterian = (x.shape[0] < self.min_samples_split
                                 or np.max(classes) / y.size >= self.sufficient_share
                                 or (classes > 0).sum() == 1
                                 or (self.max_depth and depth >= self.max_depth))
        if the_stoping_criterian:
            class_ = np.argmax(classes)
            self.tree[node_id] = [self.__class__.LEAF_TYPE, class_]
            return
            
        feature_ids = self.get_feature_ids(x.shape[1])
        gin_threshold = np.array([self.__find_threshold(x[:, i], y)
                                        for i in feature_ids])        
        if gin_threshold.size == 0:
            class_ = np.argmax(classes)
            self.tree[node_id] = [self.__class__.LEAF_TYPE, class_]
            return
        gin_ind = np.argmax(gin_threshold, axis=0)[0]
        gin, threshold = gin_threshold[gin_ind]
        feature_id = feature_ids[gin_ind]
        
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, int(feature_id), threshold)
        if x_left.size == 0 or x_right.size == 0:
            class_ = np.argmax(classes)
            self.tree[node_id] = [self.__class__.LEAF_TYPE, class_]
            return
        
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, int(feature_id), threshold]
        self.feature_importances_[feature_id] += gin
        
        self.__fit_node(x_left, y_left, node_id * 2 + 1, depth + 1)
        self.__fit_node(x_right, y_right, node_id * 2 + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(criterion='gini', min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.55 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 72 ms, sys: 4 ms, total: 76 ms
Wall time: 75.2 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8888888888888888

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv', encoding='latin1')
df = df.iloc[:, :67]
df = df.drop(['iid', 'id', 'idg', 'condtn',
         'wave', 'round', 'position', 'positin1',
         'order', 'partner', 'pid', 'pf_o_att',
         'dec_o', 'attr_o', 'imprace',
         'imprelig', 'from', 'zipcode',
         'goal', 'go_out', 'career', 'field', 'undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df['mn_sat'].fillna((df['mn_sat'].mean()), inplace=True)
df['age'].fillna((df['age'].mean()), inplace=True)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df['tuition'].fillna((df['tuition'].mean()), inplace=True)
df['career_c'].fillna(1000, inplace=True)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df['income'].fillna((df['income'].mean()), inplace=True)
df = df.dropna(axis=1)
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
features = df.columns[1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [10]:
%time clf.fit(X_train, y_train)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 17.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [11]:
%time my_clf.fit(X_train, y_train)

CPU times: user 372 ms, sys: 36 ms, total: 408 ms
Wall time: 407 ms


## Проверка качества работы на Speed Dating Data

In [12]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8124544735322837

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.947306443499337

## Задание 3

In [14]:
def top_features(clf, features):
    return features[np.argsort(clf.feature_importances_)[::-1]]

## Задание 4

In [15]:
print("my_clf: ", top_features(my_clf, features))
print("clf: ", top_features(clf, features))

my_clf:  Index(['career_c', 'income', 'mn_sat', 'age', 'tuition', 'samerace', 'match'], dtype='object')
clf:  Index(['income', 'age', 'career_c', 'mn_sat', 'tuition', 'samerace', 'match'], dtype='object')


## Задание 5

In [16]:
from sklearn.model_selection import GridSearchCV
grid_param = {
    'max_depth': np.arange(1, 5),
    'n_estimators': np.arange(10, 20, 10),
    "max_features": np.arange(1, features.size),
    'min_samples_leaf': np.arange(2, 10, 2),
    'criterion': ['entropy', 'gini']
}
clf = RandomForestClassifier()
gd_sr = GridSearchCV(estimator=clf,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1,
                     iid=True)
gd_sr.fit(X_train, y_train)
best_parameters = gd_sr.best_params_
best_score = gd_sr.best_score_
print(best_parameters)
best_score

{'criterion': 'gini', 'max_depth': 4, 'max_features': 4, 'min_samples_leaf': 4, 'n_estimators': 10}


0.7283819628647215